# Encoder scaled dot-product attention

Como hemos contado el bloque de atención está compuesto por el bloque `Scaled Dot-Prooduct Attention`

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention.png" alt="Multi-Head Attention" style="width:501px;height:623px;">
</div>

De modo que vamos a ver cómo es su arquitectura

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
</div>

Esta arquitectura se define también con la siguiente fórmula

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Así en frío puede parecer complicado, pero vamos a ir explicando todo poco a poco para entender cómo funciona este mecanismo de atención, que constituye la parte principal de la arquitectura Transformer

## Similitud entre vectores (MatMul)

La forma que tenemos de calcular la similitud entre dos vectores consiste en calcular el producto escalar entre ambos, pero esto por qué se hace? El producto escalar entre dos vectores U y V se puede descomponer entre el producto de sus normas multiplicado por el coseno del ángulo que los separa

$$\mathbf{U} \cdot \mathbf{V} = |\mathbf{U}| \cdot |\mathbf{V}| \cos(\theta)$$

Por lo que valga lo que valga el producto de sus normas, si el ángulo que los separa es de 0º, su coseno valdrá 1, lo que quiere decir que los dos vectores son similares. Si el ángulo que los separa vale 180º, su coseno valdrá -1, lo que quiere decir que los dos vectores son opuestos o antagónicos. Pero si el ángulo que los separa es de 90º o 270º, su coseno valdrá 0, por lo que los vectores no tendrán ninguna similitud. En el siguiente gif se puede ver una muestra de este comportamiento

<div style="text-align:center;">
  <img src="Imagenes/dot-product-unity.gif" alt="dot product">
</div>

Volviendo al producto escalar, en el siguiente gif podemos ver cómo a medida que cambia el ángulo entre los vectores, el producto entre ambos cambia entre el producto de sus normas, cero o el valor negativo del producto de sus normas

<div style="text-align:center;">
  <img src="Imagenes/dot-product.gif" alt="dot product">
</div>

Ya sabemos qué hace el bloque `MatMul` de `Scaled Dot-Product Attention`, ahora vamos a explicar el bloque entero y cómo consigue calcular la atención entre distintos tokens

## Scaled dot-product attention

## Atención entre tokens con y sin el efecto del positional encoding